In [ ]:
import face_recognition
import cv2
import numpy as np
import os
import glob
import pandas as pd
from datetime import datetime,timedelta
import time


In [ ]:
import tkinter.ttk as ttk
import tkinter.font as font
import tkinter as tk
from tkinter import Message, Text


unknown_names = []

def unkownvisitor(frame):
    cur_direc = os.getcwd()
    path = os.path.join(cur_direc, 'data/faces/')
    window = tk.Tk()
    window.title("Unknown face detected")
    window.geometry("1300x650")
    window.minsize(400,390)
    window.configure(background='white')
    window.grid_rowconfigure(0, weight=1)
    window.grid_columnconfigure(0, weight=1)
    message = tk.Label(
        window, text="Face-Recognition-System",
        bg="green", fg="white", width=50,
        height=3, font=('times', 30, 'bold'))

    message.place(x=200, y=20)

    lbl2 = tk.Label(window, text="first Name",
                    width=20, fg="green", bg="white",
                    height=2, font=('times', 15, ' bold '))
    lbl2.place(x=400, y=200)

    txt2 = tk.Entry(window, width=20,
                    bg="white", fg="green",
                    font=('times', 15, ' bold '))

    txt2.place(x=700, y=215)

    lbl3 = tk.Label(window, text="last Name",
                    width=20, fg="green", bg="white",
                    height=2, font=('times', 15, ' bold '))
    lbl3.place(x=400, y=300)

    txt3 = tk.Entry(window, width=20,
                    bg="white", fg="green",
                    font=('times', 15, ' bold '))

    txt3.place(x=700, y=315)
    #lbl2.place(x=400, y=300)

    #txt2.place(x=700, y=315)

    def close_window():
        window.destroy()


    def getname():
        cur_direc = os.getcwd()
        path = os.path.join(cur_direc, 'data/faces/')

        fn=txt2.get()
        ln=txt3.get()
        unknown_names.append(fn)
        unknown_names.append(ln)
        print(unknown_names)
        name(unknown_names)
        vname = tk.Label(window, text=fn,
                    width=20, fg="green", bg="white",
                    height=2, font=('times', 15, ' bold '))
        vname.place(x=400, y=300)

        close_window()

    def name(unknown_names):
        fname = unknown_names[0]
        lname = unknown_names[1]
        if fname and lname:
            img_name = "{fname}-{lname}.jpg".format(fname=fname, lname=lname)
            cv2.imwrite(str(path) + img_name, frame)
            print("{} written!".format(img_name))


    takename = tk.Button(window, text ="save",  
    command = getname, fg ="white", bg ="green",  
    width = 20, height = 3, activebackground = "Red",  
    font =('times', 15, ' bold ')) 
    unknown_names=[]

    takename.place(x = 600, y = 400) 



    window.mainloop()

In [ ]:
def markvisitor(name):

    vname=name
    temp=35
    df = pd.read_csv('Attendance.csv')
    
    now = datetime.now()

    visitor = []

 
    timenow = now.strftime('%H:%M:%S')
    
    visitor = df.query('Name == @vname and ExitTime=="inside"')
    entrytime= visitor['EntryTime']
    



    if visitor.empty:
        names_list = list(df['Name'])
        print(names_list)
        
        df.loc[len(df)] = [vname, timenow, temp, 'inside', 'none']
        
        if name in names_list:
            visitor = df.query('Name ==@name')
            exittime=visitor['ExitTime']
                               
            
    else:
        entrytime = pd.to_datetime(entrytime, format='%H:%M:%S')

        diff = datetime.strptime(timenow, '%H:%M:%S') - entrytime
        
        dts = str(diff)
        print(dts)
        dts1 = dts[14:16]
        #print(dts)
        dts2= int(dts1)
        print(dts1)
        
        if dts2 >= 2:
            df['ExitTime'] = df['ExitTime'].replace(['inside'], timenow)
            df['ExitTemp'] = df['ExitTemp'].replace(['none'], temp)
            time.sleep(5)    


    '''names_list = list(df['Name'])
    if vname not in names_list:

        df.loc[len(df)] = [vname, timenow, temp, 'inside', 'none']'''

    
    
    file = open("Attendance.csv", "r+")
    file.truncate(0)
    file.close()
    df.to_csv('Attendance.csv',index=False)

In [ ]:
def facerecog():
    
    faces_encodings = []
    faces_names = []
    cur_direc = os.getcwd()
    path = os.path.join(cur_direc, 'data/faces/')
    list_of_files = [f for f in glob.glob(path+'*.jpg')]
    number_files = len(list_of_files)

    names = []
    newnames = []
    images = []
    f_names = list_of_files.copy()
    for f_name in f_names:
        faces = f_name.split('\\')
        for name in faces:
            if name.endswith('.jpg'):
                name = name.replace('.jpg', '')
                newnames.append(name)
    names = newnames


    for i in range(number_files):
        globals()['image_{}'.format(i)] = face_recognition.load_image_file(list_of_files[i])
        globals()['image_encoding_{}'.format(i)] = face_recognition.face_encodings(globals()['image_{}'.format(i)])[0]
        faces_encodings.append(globals()['image_encoding_{}'.format(i)])

        # Create array of known names
        names[i] = names[i].replace(cur_direc,'')  
        faces_names.append(names[i])


    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True


    video_capture = cv2.VideoCapture(0)


    while True:
        ret, frame = video_capture.read()

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        rgb_small_frame = small_frame[:, :, ::-1]

        if process_this_frame:
            face_locations = face_recognition.face_locations( rgb_small_frame)
            face_encodings = face_recognition.face_encodings( rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces (faces_encodings, face_encoding)
                name = "Unknown"


                face_distances = face_recognition.face_distance( faces_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = faces_names[best_match_index]
                
                    markvisitor(name)
                    
                    
                   

                if name == "Unknown":
                    unkownvisitor(frame)
                    if cv2.waitKey(0) & 0xFF == ord('q'):
                        facerecog()
                        return
                    
                face_names.append(name)
        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Input text label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)
        

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release() 
    cv2.destroyAllWindows()

        
    
facerecog()